In [53]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

In [41]:
df = pd.read_pickle(r'D:\JupyterNotebook\DataProcess\preprocess_result2.pkl')
df.head()

,text,label
0,menpora dukung pianis cilik kendal pecah rekor...,0
1,survei bi inflasi februari persenbank indonesi...,1
2,vicky shu sih gak nikah cepatvicky shu aku lan...,2
3,kritis laku bom panci bandung tewaskabid humas...,3
4,mudah atas bau badanbagi orang bau tubuh kali ...,4


In [46]:
X = df['text']
y = df['label'].astype('int')

In [47]:
# Pembentukan tf-idf untuk pembobotan
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)
# x_test = vectorizer.transform(x_test)

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [50]:
%%time
modelknn = KNeighborsClassifier(n_neighbors=5, weights='distance')
modelknn.fit(X_train,y_train)

Wall time: 2 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='distance')

In [51]:
y_pred = modelknn.predict(X_test)
y_pred

array([ 6,  7, 10,  4,  6,  8,  4,  6,  1,  0,  2,  5,  2,  2,  7,  4,  4,
        2,  5,  6,  2,  7,  9,  0,  8,  7,  4,  6,  7,  5,  2,  8,  4,  7,
        7,  8,  3,  1,  4,  1,  8, 11,  0,  9,  5,  0,  8,  3,  2,  9,  6,
        0,  1,  8,  0,  4, 10,  2, 11, 10, 11,  6,  3,  3, 10,  6,  9,  5,
        4,  7,  3,  4])

In [52]:
modelknn.predict_proba(X_test)

array([[0.        , 0.19854254, 0.        , 0.        , 0.        ,
        0.        , 0.80145746, 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.20147884, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.79852116, 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.19491879, 0.        , 0.        , 0.        ,
        0.80508121, 0.        ],
       [0.39906966, 0.        , 0.        , 0.19905132, 0.40187902,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.82223218, 0.        , 0.        , 0.        ,
        0.        , 0.17776782],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        ,

In [54]:
print(confusion_matrix(y_test, y_pred))

[[4 0 1 0 1 0 1 1 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0 2 1]
 [1 0 7 0 0 0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0 1 0 1 0 0]
 [0 0 0 0 8 0 0 0 0 0 0 0]
 [0 0 0 0 1 4 0 0 0 0 0 0]
 [0 1 0 1 0 0 6 1 0 0 0 0]
 [1 0 0 0 0 1 0 5 0 0 0 0]
 [0 0 0 2 0 0 0 0 5 0 0 0]
 [0 0 0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 1 0 0 0 2 1]
 [0 0 0 0 0 0 0 0 2 0 0 1]]


In [55]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.50      0.57         8
           1       0.75      0.50      0.60         6
           2       0.88      0.88      0.88         8
           3       0.40      0.50      0.44         4
           4       0.80      1.00      0.89         8
           5       0.80      0.80      0.80         5
           6       0.75      0.67      0.71         9
           7       0.62      0.71      0.67         7
           8       0.71      0.71      0.71         7
           9       0.75      1.00      0.86         3
          10       0.50      0.50      0.50         4
          11       0.33      0.33      0.33         3

    accuracy                           0.69        72
   macro avg       0.66      0.68      0.66        72
weighted avg       0.70      0.69      0.69        72

